# EDA

## Purpose
The purpose of this notebook is to perform exploratory data analysis (EDA) on the training dataset to clean and prepare the dataset for classification model training. First, the data will be examined for error and null values and correction will be inferred based on majority data. Second, dummy variables will be created for binary categorical data. Last, all cleaning and feature engineering will be applied to the test data. Both data set will be saved as a new CSV.

## Data Correction and Feature Engineering Summary
1. Education: merge all non 1, 2, or 3 values to 4
2. Marital Status: merge all non 1, 2, or 3 values to 3
    - Feature Engineer this as dummy variable of married (1=True 0=False)
3. History of Payment (pay_status_month): Fix any values represented as -2 or 0. Change duly paid status of 0 to mark to complete ordinal sequence


## Column IDs

- Unnamed 0: Most likely a client ID. Not needed for this project
- X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.
- X2: Gender (1 = male; 2 = female).
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).
- X4: Marital status (1 = married; 2 = single; 3 = others).
- X5: Age (year).
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005; X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.
    - -2 probably represents overpayment or bill at 0
- X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005.
- X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005.
- Y: default payment (1=defaulted, 0=not), this is the **predictand**
    - 1 = next month will be default payment
    - 0 = not 1

## Data Import

In [1]:
# below allows local import for custom functions in src folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import src.cc_pred_tools as tl
    
import pandas as pd
pd.set_option('display.max_columns', 30) #set to show all columns

import matplotlib
import seaborn as sns
%matplotlib inline

In [2]:
directory = r"..\data\training_data.csv"  #this directory may have to change if working on MAC OS
df = pd.read_csv(directory)

In [3]:
df.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,28835,220000,2,1,2,36,0,0,0,0,0,0,222598,222168,217900,221193,181859,184605,10000,8018,10121,6006,10987,143779,1
1,25329,200000,2,3,2,29,-1,-1,-1,-1,-1,-1,326,326,326,326,326,326,326,326,326,326,326,326,0
2,18894,180000,2,1,2,27,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,690,80000,1,2,2,32,0,0,0,0,0,0,51372,51872,47593,43882,42256,42527,1853,1700,1522,1548,1488,1500,0
4,6239,10000,1,2,2,27,0,0,0,0,0,0,8257,7995,4878,5444,2639,2697,2000,1100,600,300,300,1000,1


There are no null values in this dataset

#### Column Renaming

In [4]:
df = df.drop(columns="Unnamed: 0") # drop unncessary columns

In [5]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,220000,2,1,2,36,0,0,0,0,0,0,222598,222168,217900,221193,181859,184605,10000,8018,10121,6006,10987,143779,1
1,200000,2,3,2,29,-1,-1,-1,-1,-1,-1,326,326,326,326,326,326,326,326,326,326,326,326,0
2,180000,2,1,2,27,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,80000,1,2,2,32,0,0,0,0,0,0,51372,51872,47593,43882,42256,42527,1853,1700,1522,1548,1488,1500,0
4,10000,1,2,2,27,0,0,0,0,0,0,8257,7995,4878,5444,2639,2697,2000,1100,600,300,300,1000,1


In [6]:
rename_list = ["max_credit", "gender", "education", "marital_status", "age",
               "pay_status_sep", "pay_status_aug", "pay_status_jul", "pay_status_jun", "pay_status_may", "pay_status_apr",
               "bill_sep", "bill_aug", "bill_jul", "bill_jun", "bill_may", "bill_apr",
               "payment_sep", "payment_aug", "payments_jul", "payment_jun", "payment_may", "payment_apr",
                "default"]
col_rename = dict(zip(df.columns,rename_list))

In [7]:
df = df.rename(columns=col_rename)

#### Null and Unique Values

- Check for any null values and infer new data to fill in null if necessary
- Check categorical data for their unique values to check for error

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22500 entries, 0 to 22499
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   max_credit      22500 non-null  object
 1   gender          22500 non-null  object
 2   education       22500 non-null  object
 3   marital_status  22500 non-null  object
 4   age             22500 non-null  object
 5   pay_status_sep  22500 non-null  object
 6   pay_status_aug  22500 non-null  object
 7   pay_status_jul  22500 non-null  object
 8   pay_status_jun  22500 non-null  object
 9   pay_status_may  22500 non-null  object
 10  pay_status_apr  22500 non-null  object
 11  bill_sep        22500 non-null  object
 12  bill_aug        22500 non-null  object
 13  bill_jul        22500 non-null  object
 14  bill_jun        22500 non-null  object
 15  bill_may        22500 non-null  object
 16  bill_apr        22500 non-null  object
 17  payment_sep     22500 non-null  object
 18  paymen

There are no null values in this dataset. However, checking for unique values shows that most values are integers in string form along with hidden column names.

In [9]:
category_col = ["gender", "education", "marital_status",
               "pay_status_sep", "pay_status_aug", "pay_status_jul", "pay_status_jun", "pay_status_may", "pay_status_apr",
               "default"]

In [10]:
for category in category_col:
    print(category, df[category].unique())

gender ['2' '1' 'SEX']
education ['1' '3' '2' '4' '6' '5' '0' 'EDUCATION']
marital_status ['2' '1' '3' '0' 'MARRIAGE']
pay_status_sep ['0' '-1' '-2' '2' '1' '3' '8' '4' '6' '5' '7' 'PAY_0']
pay_status_aug ['0' '-1' '-2' '2' '3' '4' '7' '5' '1' '8' '6' 'PAY_2']
pay_status_jul ['0' '-1' '-2' '2' '3' '4' '6' '5' '7' '8' '1' 'PAY_3']
pay_status_jun ['0' '-1' '-2' '2' '4' '5' '3' '7' '6' '8' '1' 'PAY_4']
pay_status_may ['0' '-1' '-2' '2' '4' '3' '7' '5' '6' '8' 'PAY_5']
pay_status_apr ['0' '-1' '-2' '2' '4' '3' '7' '6' '8' '5' 'PAY_6']
default ['1' '0' 'default payment next month']


Searching for the index with hidden column results in a single index

In [11]:
string_row_df = df[df["gender"] == "SEX"]
string_row_df.head()

,max_credit,gender,education,marital_status,age,pay_status_sep,pay_status_aug,pay_status_jul,pay_status_jun,pay_status_may,pay_status_apr,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr,default
18381,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month


This index can be dropped.

In [12]:
df = df.drop(index=string_row_df.index[0])

Running the for loop for unique values again shows that the string values are cleaned up.

In [13]:
for category in category_col:
    print(category, sorted(df[category].unique()))

gender ['1', '2']
education ['0', '1', '2', '3', '4', '5', '6']
marital_status ['0', '1', '2', '3']
pay_status_sep ['-1', '-2', '0', '1', '2', '3', '4', '5', '6', '7', '8']
pay_status_aug ['-1', '-2', '0', '1', '2', '3', '4', '5', '6', '7', '8']
pay_status_jul ['-1', '-2', '0', '1', '2', '3', '4', '5', '6', '7', '8']
pay_status_jun ['-1', '-2', '0', '1', '2', '3', '4', '5', '6', '7', '8']
pay_status_may ['-1', '-2', '0', '2', '3', '4', '5', '6', '7', '8']
pay_status_apr ['-1', '-2', '0', '2', '3', '4', '5', '6', '7', '8']
default ['0', '1']


There are few observations that are outside of the definitions.
- There are -2 and 0 within payment status columns, they might mean that the balance previous months was 0 or it has been overpaid
- There are 0, 5, and 6 for education, which might indicated no eduation or higher than masters

In order to filter out errors, all values must be changed as integers

### Change all values to int64

In [14]:
df = df.astype("int64")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22499 entries, 0 to 22499
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   max_credit      22499 non-null  int64
 1   gender          22499 non-null  int64
 2   education       22499 non-null  int64
 3   marital_status  22499 non-null  int64
 4   age             22499 non-null  int64
 5   pay_status_sep  22499 non-null  int64
 6   pay_status_aug  22499 non-null  int64
 7   pay_status_jul  22499 non-null  int64
 8   pay_status_jun  22499 non-null  int64
 9   pay_status_may  22499 non-null  int64
 10  pay_status_apr  22499 non-null  int64
 11  bill_sep        22499 non-null  int64
 12  bill_aug        22499 non-null  int64
 13  bill_jul        22499 non-null  int64
 14  bill_jun        22499 non-null  int64
 15  bill_may        22499 non-null  int64
 16  bill_apr        22499 non-null  int64
 17  payment_sep     22499 non-null  int64
 18  payment_aug     22499 non-

### Fixing Errors in Education
Changing 0, 5, 6 in education to 4

In [15]:
df.groupby("education").age.count()

education
0       11
1     7919
2    10516
3     3713
4       90
5      208
6       42
Name: age, dtype: int64

1, 2, and 3 level of education have signifcant data set. 0, 5, 6 seems to belong in 4 given how anemic this choice is

In [16]:
df.education = df.education.apply(tl.clean_education_col)

In [17]:
df.groupby("education").age.count()

education
1     7919
2    10516
3     3713
4      351
Name: age, dtype: int64

### Errors in Marital Status
Similar to education column, the unexpected value counts are extremely low (44 counts on values of 0). Therefore, all values of 0 will be changes to 3 ("others")

In [18]:
df.groupby("marital_status").age.count()

marital_status
0       44
1    10195
2    12026
3      234
Name: age, dtype: int64

In [19]:
df.marital_status = df.marital_status.apply(tl.clean_marital_status)

In [20]:
df.groupby("marital_status").age.count()

marital_status
1    10195
2    12026
3      278
Name: age, dtype: int64

### Married Feature
This column will be created using the marital_status column where 1 is given for 1 in marital_status column and 0 is given for all other columns

In [21]:
df["married"] = df.marital_status.apply(lambda x: 1 if x ==1 else 0)

In [22]:
df.head()

,max_credit,gender,education,marital_status,age,pay_status_sep,pay_status_aug,pay_status_jul,pay_status_jun,pay_status_may,pay_status_apr,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr,default,married
0,220000,2,1,2,36,0,0,0,0,0,0,222598,222168,217900,221193,181859,184605,10000,8018,10121,6006,10987,143779,1,0
1,200000,2,3,2,29,-1,-1,-1,-1,-1,-1,326,326,326,326,326,326,326,326,326,326,326,326,0,0
2,180000,2,1,2,27,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,80000,1,2,2,32,0,0,0,0,0,0,51372,51872,47593,43882,42256,42527,1853,1700,1522,1548,1488,1500,0,0
4,10000,1,2,2,27,0,0,0,0,0,0,8257,7995,4878,5444,2639,2697,2000,1100,600,300,300,1000,1,0


### Update gender column
Updating this column to 0 (male) and 1 (female)

In [23]:
df.gender = df.gender.apply(lambda x: 0 if x==1 else 1)

In [24]:
df.head()

,max_credit,gender,education,marital_status,age,pay_status_sep,pay_status_aug,pay_status_jul,pay_status_jun,pay_status_may,pay_status_apr,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr,default,married
0,220000,1,1,2,36,0,0,0,0,0,0,222598,222168,217900,221193,181859,184605,10000,8018,10121,6006,10987,143779,1,0
1,200000,1,3,2,29,-1,-1,-1,-1,-1,-1,326,326,326,326,326,326,326,326,326,326,326,326,0,0
2,180000,1,1,2,27,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,80000,0,2,2,32,0,0,0,0,0,0,51372,51872,47593,43882,42256,42527,1853,1700,1522,1548,1488,1500,0,0
4,10000,0,2,2,27,0,0,0,0,0,0,8257,7995,4878,5444,2639,2697,2000,1100,600,300,300,1000,1,0


# THIS IS ONLY USED TO CHECK AND VALIDATE FEATURE ENGINEERING

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [26]:
x = df.drop(columns="default")
y = df.default

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42)

In [31]:
logreg = LogisticRegression(max_iter = 10**4)
logreg.fit(x_train, y_train)


LogisticRegression(max_iter=10000)

In [33]:
y_pred= logreg.predict(x_test)

In [34]:
metrics.f1_score(y_test, y_pred)

0.0

# NOT SURE ABOUT BELOW IMPLEMENTATION
***

### Errors in pay_status columns

These columns contain categorical values representing the payment status. The value is supposed to range from 1-9 depending how late the payments are or marked a -1 if they are duly paid. However, there are -2 as a value within these columns, which may indicate over payment. Checking for 0 bills show that most payment status are set as -2, some are marked as 1 (1 month delay in payment) even when the bills are due at 0. These pay status will be changed to 0.

In [21]:
df[(df.bill_sep==0) & (df.bill_aug==0) & (df.bill_jul==0) &
   (df.bill_jun==0) & (df.bill_may==0) & (df.bill_apr==0) ].mean()

max_credit        213771.251932
gender                 1.633694
education              1.672334
marital_status         1.472952
age                   36.840804
pay_status_sep        -0.085008
pay_status_aug        -2.000000
pay_status_jul        -2.000000
pay_status_jun        -2.000000
pay_status_may        -2.000000
pay_status_apr        -2.000000
bill_sep               0.000000
bill_aug               0.000000
bill_jul               0.000000
bill_jun               0.000000
bill_may               0.000000
bill_apr               0.000000
payment_sep           10.046368
payment_aug          154.559505
payments_jul           0.000000
payment_jun           15.455951
payment_may            0.000000
payment_apr          305.947450
default                0.346213
dtype: float64

In [22]:
df[(df.bill_sep==0) & (df.bill_aug==0) & (df.bill_jul==0) &
   (df.bill_jun==0) & (df.bill_may==0) & (df.bill_apr==0) ]

,max_credit,gender,education,marital_status,age,pay_status_sep,pay_status_aug,pay_status_jul,pay_status_jun,pay_status_may,pay_status_apr,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr,default
2,180000,2,1,2,27,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
22,120000,2,2,2,28,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
59,360000,2,1,1,42,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
71,350000,2,1,2,26,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
73,160000,2,1,2,40,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22384,180000,2,1,1,28,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
22433,200000,2,1,1,34,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
22474,360000,2,1,1,49,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
22480,210000,2,1,2,30,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
pay_status_columns = ["pay_status_sep", "pay_status_aug", "pay_status_jul", 
                      "pay_status_jun", "pay_status_may", "pay_status_apr"]

for stat_month in pay_status_columns:
    df[stat_month] = df[stat_month].apply(tl.clean_pay_status)

In [24]:
df[((df.pay_status_sep==0) & (df.pay_status_aug==0) & (df.pay_status_jul==0) &
   (df.pay_status_jun==0) & (df.pay_status_may==0) & (df.pay_status_apr==0))]

,max_credit,gender,education,marital_status,age,pay_status_sep,pay_status_aug,pay_status_jul,pay_status_jun,pay_status_may,pay_status_apr,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr,default
0,220000,2,1,2,36,0,0,0,0,0,0,222598,222168,217900,221193,181859,184605,10000,8018,10121,6006,10987,143779,1
1,200000,2,3,2,29,0,0,0,0,0,0,326,326,326,326,326,326,326,326,326,326,326,326,0
2,180000,2,1,2,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,80000,1,2,2,32,0,0,0,0,0,0,51372,51872,47593,43882,42256,42527,1853,1700,1522,1548,1488,1500,0
4,10000,1,2,2,27,0,0,0,0,0,0,8257,7995,4878,5444,2639,2697,2000,1100,600,300,300,1000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22487,30000,2,2,2,21,0,0,0,0,0,0,26757,26831,26620,26134,26865,25752,1520,1339,852,2067,1400,100,0
22488,90000,2,2,2,23,0,0,0,0,0,0,42105,43919,46812,44976,46406,47625,2500,3970,2500,2500,2500,1700,1
22493,180000,2,1,2,28,0,0,0,0,0,0,171237,172787,174667,176618,141697,143738,8000,7000,7000,5010,5200,6000,0
22496,350000,1,1,1,42,0,0,0,0,0,0,3800,3138,4150,3750,1362,8210,3138,4160,3750,2272,8210,9731,0


In [36]:
df[((df.bill_sep==0) | (df.bill_aug==0) | (df.bill_jul==0) |
   (df.bill_jun==0) | (df.bill_may==0) | (df.bill_apr==0)) &
   ((df.pay_status_sep==-2) | (df.pay_status_aug==-2) | (df.pay_status_jul==-2) |
   (df.pay_status_jun==-2) | (df.pay_status_may==-2) | (df.pay_status_apr==-2))]

,max_credit,gender,education,marital_status,age,pay_status_sep,pay_status_aug,pay_status_jul,pay_status_jun,pay_status_may,pay_status_apr,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr,default
2,180000,2,1,2,27,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
22,120000,2,2,2,28,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
26,80000,2,2,1,45,0,0,-2,-2,-2,-2,82150,0,0,0,0,0,0,0,0,0,0,0,0
57,260000,2,1,2,33,-2,-2,-1,-1,-1,0,1900,0,204,-810,2190,810,0,204,0,3000,0,0,0
58,360000,2,1,2,35,0,0,-1,0,0,-2,6392,3773,17917,21057,0,0,3000,18117,5371,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22478,30000,1,2,2,28,0,0,0,0,0,-2,26054,25144,25775,18585,0,0,2000,2000,2000,0,0,0,0
22480,210000,2,1,2,30,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
22483,360000,2,1,1,45,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0
22491,10000,2,2,2,22,1,2,0,0,0,-2,7194,6934,8786,8014,0,0,0,2000,430,0,0,0,1


what does it mean when there are 0 bills but a payment is made? Also if there are no bills how could they have default?